# Stage 7 — Interactive Stress Test Dashboard

This notebook provides an interactive dashboard to explore the Stage 6 stress test results.
It loads the aggregated `stress_test_summary.csv` (created by `scripts/stress_tester.py`) and the per-regime `portfolio_value.csv` files from each `results/versions/run_*` folder and provides interactive visualizations (Plotly) and quick metric displays.

---




In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

PROJECT_ROOT = Path(r"D:\Portfolio Optimzation project")
BASE_DIR = PROJECT_ROOT / "results" / "versions"
SUMMARY_FILE = BASE_DIR / "stress_test_summary.csv"

print(f"Using BASE_DIR = {BASE_DIR}")

Using BASE_DIR = D:\Portfolio Optimzation project\results\versions


In [3]:
summary = pd.read_csv(SUMMARY_FILE, index_col=0)
summary.index.name = 'Scenario'
summary = summary.sort_index()
summary

,CAGR,Volatility,Sharpe,Sortino,Calmar,VaR(95%),CVaR(95%),MaxDrawdown
Scenario,,,,,,,,
Bear Market 2020,6.9363,0.3636,5.7198,28.5436,237.9755,-0.0204,-0.0282,-0.0291
Sideways Market 2021 2022,0.3430,0.1770,1.6670,2.2875,1.8375,-0.0195,-0.0280,-0.1867


In [4]:
fig = px.imshow(summary.T, text_auto=True, aspect="auto",
                labels=dict(x="Scenario", y="Metric", color="Value"))
fig.update_layout(title_text="Stress Test Metrics Heatmap (per regime)")
fig.show()

In [5]:
from plotly.subplots import make_subplots
fig = make_subplots(rows=4, cols=1, shared_xaxes=True,
                    subplot_titles=("CAGR", "Volatility", "Max Drawdown", "Risk-Adjusted Ratios (Sharpe, Sortino, Calmar)"))

fig.add_trace(go.Bar(x=summary.index, y=summary['CAGR'], name='CAGR', marker_color='steelblue'), row=1, col=1)
fig.add_trace(go.Bar(x=summary.index, y=summary['Volatility'], name='Volatility', marker_color='orange'), row=2, col=1)
fig.add_trace(go.Bar(x=summary.index, y=summary['MaxDrawdown'], name='MaxDrawdown', marker_color='red'), row=3, col=1)
fig.add_trace(go.Bar(x=summary.index, y=summary['Sharpe'], name='Sharpe'), row=4, col=1)
fig.add_trace(go.Bar(x=summary.index, y=summary['Sortino'], name='Sortino'), row=4, col=1)
fig.add_trace(go.Bar(x=summary.index, y=summary['Calmar'], name='Calmar'), row=4, col=1)

fig.update_layout(height=1100, showlegend=True, title_text="Stress Test Summary — Key Metrics")
fig.update_xaxes(tickangle=25)
fig.show()

In [ ]:
# pip install ipywidgets

In [6]:
pv_map = {}
for run in BASE_DIR.glob('run_*'):
    name = run.name.replace('run_', '').replace('_', ' ')
    pv_file = run / 'portfolio_value.csv'
    if pv_file.exists():
        df = pd.read_csv(pv_file, parse_dates=['date'])
        val_col = [c for c in df.columns if c != 'date'][0]
        s = df.set_index('date')[val_col].sort_index()
        pv_map[name] = s

import ipywidgets as widgets
from IPython.display import display, clear_output

scenarios = list(pv_map.keys())
selector = widgets.Dropdown(options=scenarios, description='Scenario:')
output = widgets.Output()

def show_scenario(change):
    with output:
        clear_output()
        scen = change['new']
        s = pv_map[scen]
        fig = px.line(x=s.index, y=s.values, labels={'x':'Date','y':'Portfolio Value'}, title=f'Cumulative Portfolio Value — {scen}')
        fig.update_traces(line=dict(width=2))
        fig.show()

selector.observe(show_scenario, names='value')
display(selector)
display(output)
selector.value = scenarios[0] if scenarios else None

Dropdown(description='Scenario:', options=('2025-10-24 02-01', '2025-10-24 02-12', '2025-10-25 14-11', 'Bear M…

Output()

In [7]:
EXPORT_DIR = BASE_DIR / 'stress_test_report'
EXPORT_DIR.mkdir(exist_ok=True)
summary.to_csv(EXPORT_DIR / 'stress_test_summary.csv')

for scen, s in pv_map.items():
    s.to_frame('portfolio_value').to_csv(EXPORT_DIR / f'portfolio_value_{scen.replace(" ","_")}.csv')

print('Exported summary and per-scenario portfolio values to', EXPORT_DIR)

Exported summary and per-scenario portfolio values to D:\Portfolio Optimzation project\results\versions\stress_test_report
